<a href="https://colab.research.google.com/github/Hardik2206024/Uber-app-based-self-made-Low-fidelity-wireframe/blob/main/English%20to%20Odia%20through%20Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ Install dependencies (Colab/local)
!pip install transformers datasets sacrebleu sentencepiece

import pandas as pd
from datasets import Dataset
from transformers import MarianTokenizer, MarianMTModel, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import sacrebleu

# 1. Load dataset (handle encoding properly)
file_path = "/content/engtoodia.csv"  # CSV with columns: English, Odia
try:
    df = pd.read_csv(file_path, encoding="utf-8-sig")
except:
    df = pd.read_csv(file_path, encoding="utf-16")

# Rename columns if needed
df = df.rename(columns={df.columns[0]: "English", df.columns[1]: "Odia"})

# 2. Convert to HuggingFace dataset
dataset = Dataset.from_pandas(df)
train_test = dataset.train_test_split(test_size=0.1, seed=42)

# 3. Load pre-trained model & tokenizer (English → Many languages)
model_name = "Helsinki-NLP/opus-mt-en-mul"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pr_


ParserError: Error tokenizing data. C error: Expected 1 fields in line 1405, saw 3


In [ ]:
df = pd.read_csv(file_path, encoding=result["encoding"], sep=",", on_bad_lines="skip")


NameError: name 'result' is not defined

In [ ]:
# ✅ Install dependencies
!pip install transformers datasets sacrebleu sentencepiece

import pandas as pd
from datasets import Dataset
from transformers import MarianTokenizer, MarianMTModel
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import sacrebleu

# 1. Load dataset (UTF-16 LE)
file_path = "/content/engtoodia.csv"  # Ensure file is uploaded
df = pd.read_csv(file_path, encoding="utf-16", sep="\t", on_bad_lines="skip")

# Rename columns to standard names
df = df.rename(columns={df.columns[0]: "English", df.columns[1]: "Odia"})

# 2. Convert to HuggingFace dataset
dataset = Dataset.from_pandas(df)
train_test = dataset.train_test_split(test_size=0.1, seed=42)

# 3. Load pre-trained model & tokenizer (English → Multilingual)
model_name = "Helsinki-NLP/opus-mt-en-mul"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# 4. Preprocess function
def preprocess(batch):
    inputs = tokenizer(batch["English"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(batch["Odia"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

train_data = train_test["train"].map(preprocess, batched=True)
test_data = train_test["test"].map(preprocess, batched=True)

# 5. Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="/content/logs",
    logging_steps=50,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# 6. BLEU computation with sacrebleu
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    bleu = sacrebleu.corpus_bleu(decoded_preds, [decoded_labels])
    return {"bleu": bleu.score}

# 7. Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 8. Train
trainer.train()

# 9. Evaluate
metrics = trainer.evaluate()
print("Final BLEU Score:", metrics["eval_bleu"])

# 10. Show some predictions
preds = trainer.predict(test_data)
decoded_preds = tokenizer.batch_decode(preds.predictions, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(preds.label_ids, skip_special_tokens=True)

print("\nSample Predictions:\n")
for i in range(5):
    print(f"Original English : {test_data['English'][i]}")
    print(f"Expected Odia   : {decoded_labels[i]}")
    print(f"Predicted Odia  : {decoded_preds[i]}")
    print("-" * 60)


IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
import pandas as pd

file_path = "/content/engtoodia.csv"

# Try reading without forcing sep
df_raw = pd.read_csv(file_path, encoding="utf-16", on_bad_lines="skip", header=None)
print(df_raw.head(10))
print("Number of columns detected:", df_raw.shape[1])

                                       0
0               video ad||ଭିଡିଓ ବିଜ୍ଞାପନ
1                      privacy||ଗୋପନୀୟତା
2                          unread||ଅପଠିତ
3          recharge shop||ରିଚାର୍ଜ୍ ଦୋକାନ
4         preset study||ପ୍ରିସେଟ୍‌ ଅଧ୍ୟୟନ
5            default ad||ଡିଫଲ୍ଟ ବିଜ୍ଞାପନ
6  weighted split||ଗୁରୁତ୍ୱପୂର୍ଣ୍ଣ ବିଭାଜନ
7          Ads Manager||ବିଜ୍ଞାପନ ପରିଚାଳକ
8               customize||ଅନୁକୂଳ କରାଇବା
9        creative concept||ସୃଜନଶୀଳ ଧାରଣା
Number of columns detected: 1


In [ ]:
import pandas as pd

file_path = "/content/engtoodia.csv"

# Read as single column
df_raw = pd.read_csv(file_path, encoding="utf-16", header=None, names=["pair"])

# Split English and Odia using "||"
df = df_raw["pair"].str.split("||", n=1, expand=True)
df.columns = ["English", "Odia"]

print(df.head())


ParserError: Error tokenizing data. C error: Expected 1 fields in line 1405, saw 3


In [ ]:
import pandas as pd

file_path = "/content/engtoodia.csv"

# Read raw lines safely with UTF-16
with open(file_path, "r", encoding="utf-16") as f:
    lines = f.readlines()

# Split only on the first occurrence of "||"
data = [line.strip().split("||", 1) for line in lines if "||" in line]

# Convert to DataFrame
df = pd.DataFrame(data, columns=["English", "Odia"])

print(df.head())
print("Total pairs:", len(df))


         English              Odia
0       video ad    ଭିଡିଓ ବିଜ୍ଞାପନ
1        privacy          ଗୋପନୀୟତା
2         unread             ଅପଠିତ
3  recharge shop    ରିଚାର୍ଜ୍ ଦୋକାନ
4   preset study  ପ୍ରିସେଟ୍‌ ଅଧ୍ୟୟନ
Total pairs: 1821


In [ ]:
# ✅ Install dependencies
!pip install transformers datasets sacrebleu sentencepiece

import pandas as pd
from datasets import Dataset
from transformers import MarianTokenizer, MarianMTModel
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import sacrebleu

# 1. Load dataset (UTF-16 LE + manual split)
file_path = "/content/engtoodia.csv"

with open(file_path, "r", encoding="utf-16") as f:
    lines = f.readlines()

# Split English-Odia on first "||"
data = [line.strip().split("||", 1) for line in lines if "||" in line]
df = pd.DataFrame(data, columns=["English", "Odia"])
print("Total pairs:", len(df))
print(df.head())

# 2. Convert to HuggingFace dataset
dataset = Dataset.from_pandas(df)
train_test = dataset.train_test_split(test_size=0.1, seed=42)

# 3. Load pre-trained model & tokenizer (English → Multilingual)
model_name = "Helsinki-NLP/opus-mt-en-mul"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# 4. Preprocess function
def preprocess(batch):
    inputs = tokenizer(batch["English"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(batch["Odia"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

train_data = train_test["train"].map(preprocess, batched=True)
test_data = train_test["test"].map(preprocess, batched=True)

# 5. Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="/content/logs",
    logging_steps=50,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# 6. BLEU computation with sacrebleu
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    bleu = sacrebleu.corpus_bleu(decoded_preds, [decoded_labels])
    return {"bleu": bleu.score}

# 7. Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# 8. Train
trainer.train()

# 9. Evaluate
metrics = trainer.evaluate()
print("Final BLEU Score:", metrics["eval_bleu"])

# 10. Show some predictions
preds = trainer.predict(test_data)
decoded_preds = tokenizer.batch_decode(preds.predictions, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(preds.label_ids, skip_special_tokens=True)

print("\nSample Predictions:\n")
for i in range(5):
    print(f"Original English : {test_data['English'][i]}")
    print(f"Expected Odia   : {decoded_labels[i]}")
    print(f"Predicted Odia  : {decoded_preds[i]}")
    print("-" * 60)


Total pairs: 1821
         English              Odia
0       video ad    ଭିଡିଓ ବିଜ୍ଞାପନ
1        privacy          ଗୋପନୀୟତା
2         unread             ଅପଠିତ
3  recharge shop    ରିଚାର୍ଜ୍ ଦୋକାନ
4   preset study  ପ୍ରିସେଟ୍‌ ଅଧ୍ୟୟନ


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/1638 [00:00<?, ? examples/s]

Map:   0%|          | 0/183 [00:00<?, ? examples/s]

TypeError: Seq2SeqTrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [ ]:
trainer.save_model("/content/eng-odia-model")
tokenizer.save_pretrained("/content/eng-odia-model")


('/content/eng-odia-model/tokenizer_config.json',
 '/content/eng-odia-model/special_tokens_map.json',
 '/content/eng-odia-model/vocab.json',
 '/content/eng-odia-model/source.spm',
 '/content/eng-odia-model/target.spm',
 '/content/eng-odia-model/added_tokens.json')

In [ ]:
from transformers import pipeline

# Load trained model
translation_pipeline = pipeline("translation", model="/content/eng-odia-model", tokenizer="/content/eng-odia-model")

# Take a few examples from dataset
test_samples = df.sample(5, random_state=42)  # 5 random examples

results = []
for _, row in test_samples.iterrows():
    src_text = row["English"]
    tgt_text = row["Odia"]
    pred = translation_pipeline(src_text, max_length=50)[0]['translation_text']
    results.append({"Original": src_text, "Predicted": pred, "Expected": tgt_text})

import pandas as pd
pd.DataFrame(results)


/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


,Original,Predicted,Expected
0,door lock,>>ori<<ାର ଲକ୍‌,ଦ୍ୱାର ତାଲା
1,Jack Churchill,>>ori<<ିଲ୍‌ ଜ୍ୟାକ୍‌ ଚର୍ଚିଲ୍‌,ଜ୍ୟାକ ଚର୍ଚିଲ
2,People Sets,>>ori<<ାର ସେଟ୍‌‌‌‌‌‌‌‌‌‌‌‌‌‌,ଲୋକଙ୍କ ସମାହାର
3,squeeze,>>ori<<ାଡ଼ିବା,ସ୍କୁଇଜ୍‌
4,Confirm clear,>>ori<<ାରଣ କରିବାକୁ ନିଶ୍ଚିତ କରନ୍ତୁ,ସଫାକରିବାକୁ ନିଷ୍ଚିତ କରନ୍ତୁ


In [ ]:
import evaluate

bleu = evaluate.load("sacrebleu")

# Collect references and predictions
preds = []
refs = []

for _, row in df.sample(100, random_state=0).iterrows():  # test on 100 samples
    src_text = row["English"]
    tgt_text = row["Odia"]
    pred = translation_pipeline(src_text, max_length=50)[0]['translation_text']
    preds.append(pred)
    refs.append([tgt_text])  # sacrebleu expects list of references

bleu_score = bleu.compute(predictions=preds, references=refs)
print("BLEU Score:", bleu_score)


BLEU Score: {'score': 1.880507076770648, 'counts': [57, 12, 5, 4], 'totals': [736, 636, 536, 436], 'precisions': [7.744565217391305, 1.8867924528301887, 0.9328358208955224, 0.9174311926605505], 'bp': 1.0, 'sys_len': 736, 'ref_len': 244}


In [ ]:
# If needed (e.g., in Colab), install deps:
# !pip install transformers datasets sacrebleu sentencepiece torch --quiet

import os
import pandas as pd
import torch
from datasets import Dataset
from transformers import MarianTokenizer, MarianMTModel, Seq2SeqTrainingArguments, Seq2SeqTrainer
import sacrebleu

# ---------------------------
# 1) Load & prepare the data
# ---------------------------
file_path = "/content/engtoodia.csv"

# Read raw lines (UTF-16 LE) and split on the FIRST "||"
with open(file_path, "r", encoding="utf-16") as f:
    lines = f.readlines()

pairs = [line.rstrip("\n").split("||", 1) for line in lines if "||" in line]
df = pd.DataFrame(pairs, columns=["English", "Odia"])
print("Total pairs:", len(df))
print(df.head())

# Build HF Dataset & split
dataset = Dataset.from_pandas(df)
split = dataset.train_test_split(test_size=0.1, seed=42)
train_ds = split["train"]
test_ds  = split["test"]

# ---------------------------
# 2) Load model & tokenizer
# ---------------------------
model_name = "Helsinki-NLP/opus-mt-en-mul"   # EN -> many langs (we fine-tune to Odia)
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Try to force Odia BOS token if available (helps generation)
odia_tokens = [">>or<<", ">>ori<<", ">>ory<<"]
vocab = tokenizer.get_vocab()
for tkn in odia_tokens:
    if tkn in vocab:
        model.config.forced_bos_token_id = tokenizer.convert_tokens_to_ids(tkn)
        break

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# ---------------------------
# 3) Tokenize
# ---------------------------
max_len = 128

def preprocess(batch):
    # Inputs
    model_inputs = tokenizer(
        batch["English"],
        max_length=max_len,
        truncation=True,
        padding="max_length",
    )
    # Labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["Odia"],
            max_length=max_len,
            truncation=True,
            padding="max_length",
        )["input_ids"]

    # Replace padding token id's in labels by -100 so they are ignored in loss
    pad_id = tokenizer.pad_token_id
    labels = [[(lid if lid != pad_id else -100) for lid in label] for label in labels]
    model_inputs["labels"] = labels
    return model_inputs

train_tokenized = train_ds.map(preprocess, batched=True, remove_columns=train_ds.column_names)
test_tokenized  = test_ds.map(preprocess,  batched=True, remove_columns=test_ds.column_names)

# Set format for PyTorch
train_tokenized.set_format(type="torch")
test_tokenized.set_format(type="torch")

# ---------------------------
# 4) Training arguments
#    (no evaluation_strategy — compatible with older transformers)
# ---------------------------
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    num_train_epochs=3,
    logging_dir="/content/logs",
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    # fp16 speeds things up on GPU; safe to keep off on CPU
    fp16=torch.cuda.is_available(),
)

# ---------------------------
# 5) Trainer (no DataCollator dependency)
# ---------------------------
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,  # will evaluate at the end via trainer.evaluate()
    tokenizer=tokenizer,
)

# ---------------------------
# 6) Train
# ---------------------------
trainer.train()

# ---------------------------
# 7) Evaluate: Generate translations and compute BLEU
#    (avoid predict_with_generate; use model.generate directly)
# ---------------------------
model.eval()

# Recover raw texts for test set
test_english = test_ds["English"]
test_odia_gt = test_ds["Odia"]

preds = []
for src in test_english:
    enc = tokenizer(src, return_tensors="pt", truncation=True, max_length=max_len).to(device)
    with torch.no_grad():
        gen = model.generate(
            **enc,
            max_length=max_len,
            num_beams=4,
            early_stopping=True
        )
    pred = tokenizer.decode(gen[0], skip_special_tokens=True)
    preds.append(pred)

bleu = sacrebleu.corpus_bleu(preds, [test_odia_gt]).score
print(f"\nFinal BLEU score: {bleu:.2f}")

# ---------------------------
# 8) Show samples: original | expected | predicted
# ---------------------------
print("\nSample Predictions (first 10):\n")
for i in range(min(10, len(test_english))):
    print(f"Original : {test_english[i]}")
    print(f"Expected : {test_odia_gt[i]}")
    print(f"Predicted: {preds[i]}")
    print("-" * 60)

# ---------------------------
# 9) (Optional) Save fine-tuned model
# ---------------------------
# save_dir = "/content/odia_translator_finetuned"
# os.makedirs(save_dir, exist_ok=True)
# trainer.save_model(save_dir)
# tokenizer.save_pretrained(save_dir)
# print(f"Saved fine-tuned model to: {save_dir}")


Total pairs: 1821
         English              Odia
0       video ad    ଭିଡିଓ ବିଜ୍ଞାପନ
1        privacy          ଗୋପନୀୟତା
2         unread             ଅପଠିତ
3  recharge shop    ରିଚାର୍ଜ୍ ଦୋକାନ
4   preset study  ପ୍ରିସେଟ୍‌ ଅଧ୍ୟୟନ


/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/1638 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/183 [00:00<?, ? examples/s]

/tmp/ipython-input-2226507912.py:106: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

KeyboardInterrupt: 

In [ ]:
# If needed (e.g., in Colab), install deps:
# !pip install transformers datasets sacrebleu sentencepiece torch --quiet

import os
import pandas as pd
import torch
from datasets import Dataset
from transformers import MarianTokenizer, MarianMTModel, Seq2SeqTrainingArguments, Seq2SeqTrainer
import sacrebleu

# ---------------------------
# 1) Load & prepare the data
# ---------------------------
file_path = "/content/engtoodia.csv"

# Read raw lines (UTF-16 LE) and split on the FIRST "||"
with open(file_path, "r", encoding="utf-16") as f:
    lines = f.readlines()

pairs = [line.rstrip("\n").split("||", 1) for line in lines if "||" in line]
df = pd.DataFrame(pairs, columns=["English", "Odia"])
print("Total pairs:", len(df))
print(df.head())

# Build HF Dataset & split
dataset = Dataset.from_pandas(df)
split = dataset.train_test_split(test_size=0.1, seed=42)
train_ds = split["train"]
test_ds  = split["test"]

# ---------------------------
# 2) Load model & tokenizer
# ---------------------------
model_name = "Helsinki-NLP/opus-mt-en-mul"   # EN -> many langs (we fine-tune to Odia)
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Try to force Odia BOS token if available (helps generation)
odia_tokens = [">>or<<", ">>ori<<", ">>ory<<"]
vocab = tokenizer.get_vocab()
for tkn in odia_tokens:
    if tkn in vocab:
        model.config.forced_bos_token_id = tokenizer.convert_tokens_to_ids(tkn)
        break

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# ---------------------------
# 3) Tokenize
# ---------------------------
max_len = 128

def preprocess(batch):
    # Inputs
    model_inputs = tokenizer(
        batch["English"],
        max_length=max_len,
        truncation=True,
        padding="max_length",
    )
    # Labels
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["Odia"],
            max_length=max_len,
            truncation=True,
            padding="max_length",
        )["input_ids"]

    # Replace padding token id's in labels by -100 so they are ignored in loss
    pad_id = tokenizer.pad_token_id
    labels = [[(lid if lid != pad_id else -100) for lid in label] for label in labels]
    model_inputs["labels"] = labels
    return model_inputs

train_tokenized = train_ds.map(preprocess, batched=True, remove_columns=train_ds.column_names)
test_tokenized  = test_ds.map(preprocess,  batched=True, remove_columns=test_ds.column_names)

# Set format for PyTorch
train_tokenized.set_format(type="torch")
test_tokenized.set_format(type="torch")

# ---------------------------
# 4) Training arguments
#    (no evaluation_strategy — compatible with older transformers)
# ---------------------------
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    num_train_epochs=3,
    logging_dir="/content/logs",
    logging_steps=50,
    save_steps=500,
    save_total_limit=2,
    # fp16 speeds things up on GPU; safe to keep off on CPU
    fp16=torch.cuda.is_available(),
)

# ---------------------------
# 5) Trainer (no DataCollator dependency)
# ---------------------------
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,  # will evaluate at the end via trainer.evaluate()
    tokenizer=tokenizer,
)

# ---------------------------
# 6) Train
# ---------------------------
trainer.train()

# ---------------------------
# 7) Evaluate: Generate translations and compute BLEU
#    (avoid predict_with_generate; use model.generate directly)
# ---------------------------
model.eval()

# Recover raw texts for test set
test_english = test_ds["English"]
test_odia_gt = test_ds["Odia"]

preds = []
for src in test_english:
    enc = tokenizer(src, return_tensors="pt", truncation=True, max_length=max_len).to(device)
    with torch.no_grad():
        gen = model.generate(
            **enc,
            max_length=max_len,
            num_beams=4,
            early_stopping=True
        )
    pred = tokenizer.decode(gen[0], skip_special_tokens=True)
    preds.append(pred)

bleu = sacrebleu.corpus_bleu(preds, [test_odia_gt]).score
print(f"\nFinal BLEU score: {bleu:.2f}")

# ---------------------------
# 8) Show samples: original | expected | predicted
# ---------------------------
print("\nSample Predictions (first 10):\n")
for i in range(min(10, len(test_english))):
    print(f"Original : {test_english[i]}")
    print(f"Expected : {test_odia_gt[i]}")
    print(f"Predicted: {preds[i]}")
    print("-" * 60)

# ---------------------------
# 9) (Optional) Save fine-tuned model
# ---------------------------
# save_dir = "/content/odia_translator_finetuned"
# os.makedirs(save_dir, exist_ok=True)
# trainer.save_model(save_dir)
# tokenizer.save_pretrained(save_dir)
# print(f"Saved fine-tuned model to: {save_dir}")


Total pairs: 1821
         English              Odia
0       video ad    ଭିଡିଓ ବିଜ୍ଞାପନ
1        privacy          ଗୋପନୀୟତା
2         unread             ଅପଠିତ
3  recharge shop    ରିଚାର୍ଜ୍ ଦୋକାନ
4   preset study  ପ୍ରିସେଟ୍‌ ଅଧ୍ୟୟନ


/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Map:   0%|          | 0/1638 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/183 [00:00<?, ? examples/s]

...

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
50,2.752700
100,1.850700
150,1.673500
200,1.651400


Step,Training Loss
50,2.752700
100,1.850700
150,1.673500
200,1.651400
250,1.433000
300,1.368100
350,1.332000
400,1.258300
450,1.172700
500,1.195500


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:4034: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[64109]], 'forced_bos_token_id': 898}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(



Final BLEU score: 1.02

Sample Predictions (first 10):

Original : interstitial ad
Expected : ଅନ୍ତରାଳୀୟ ବିଜ୍ଞାପନ
Predicted: >>ori<<ିଷ୍ଠାଯୁକ୍ତ ବିଜ୍ଞାପନ
------------------------------------------------------------
Original : ad breaks
Expected : ବିଜ୍ଞାପନ ବିରତିଗୁଡିକ
Predicted: >>ori<<ଣ୍ଡ ବିରତି
------------------------------------------------------------
Original : publisher list
Expected : ପ୍ରକାଶକ ତାଲିକା
Predicted: >>ori<<ର୍ତ୍ତା ତାଲିକା
------------------------------------------------------------
Original : memory card
Expected : ମେମୋରୀ କାର୍ଡ 
Predicted: >>ori<<ୃତ୍ୟ କାର୍ଡ
------------------------------------------------------------
Original : delete
Expected : ବିଲୋପ କରନ୍ତୁ
Predicted: >>ori<<ାଉଛି
------------------------------------------------------------
Original : universal creative
Expected : ସାର୍ବଜନୀନ ସୃଜନଶୀଳ
Predicted: >>ori<<ୁମାନିକ ସୃଷ୍ଟିକରଣ
------------------------------------------------------------
Original : Ad Staging
Expected : ବିଜ୍ଞାପନ ଷ୍ଟେଜିଂ
Predicted: >>ori<<ାର୍ଦ୍ଧିକରଣ
---

In [ ]:
pip install --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 20.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.4
    Uninstalling transformers-4.55.4:
      Successfully uninstalled transformers-4.55.4


In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/results",
    # evaluation_strategy="epoch",   ❌ remove this
    eval_steps=500,                  # ✅ evaluate every 500 steps instead
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="/content/logs",
    logging_steps=50,
)


In [ ]:
!pip install evaluate sacrebleu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.9 MB/s eta 0:00:00
